In [3]:
#Download Annotations
!wget -O train_annotations.txt https://s3.amazonaws.com/kinetics/600/annotations/train.txt
!wget -O val_annotations.txt https://s3.amazonaws.com/kinetics/600/annotations/val.txt
!wget -O test_annotations.csv https://s3.amazonaws.com/kinetics/600/annotations/test.csv

--2025-02-26 21:06:31--  https://s3.amazonaws.com/kinetics/600/annotations/train.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.228.112, 54.231.164.208, 52.216.251.166, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.228.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12849665 (12M) [text/plain]
Saving to: ‘train_annotations.txt’

train_annotations.t 100%[===================>]  12.25M  2.21MB/s    in 12s     

2025-02-26 21:06:45 (1019 KB/s) - ‘train_annotations.txt’ saved [12849665/12849665]

--2025-02-26 21:06:45--  https://s3.amazonaws.com/kinetics/600/annotations/val.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 16.15.176.90, 54.231.139.160, 54.231.198.176, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|16.15.176.90|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 915755 (894K) [text/plain]
Saving to: ‘val_annotations.txt’

val_annotations.txt 100%[===================>] 894.29K   816KB/s 

In [9]:
!wget -O k600_val_paths.txt https://s3.amazonaws.com/kinetics/600/val/k600_val_path.txt


--2025-02-26 21:08:53--  https://s3.amazonaws.com/kinetics/600/val/k600_val_path.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.217.40, 3.5.13.30, 3.5.16.101, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.217.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38609 (38K) [text/plain]
Saving to: ‘k600_val_paths.txt’

k600_val_paths.txt  100%[===================>]  37.70K   172KB/s    in 0.2s    

2025-02-26 21:08:54 (172 KB/s) - ‘k600_val_paths.txt’ saved [38609/38609]



In [13]:
# Count total lines (each line is a file path)
#!wc -l k600_val_paths.txt


600 k600_val_paths.txt


In [15]:
#!head -n 100 k600_val_paths.txt > k600_val_subset.txt

In [17]:
with open("k600_val_paths.txt", "r") as f:
    for i in range(5):  # Print first 5 lines
        print(f.readline().strip())


https://s3.amazonaws.com/kinetics/600/val/abseiling.tar.gz
https://s3.amazonaws.com/kinetics/600/val/acting in play.tar.gz
https://s3.amazonaws.com/kinetics/600/val/adjusting glasses.tar.gz
https://s3.amazonaws.com/kinetics/600/val/air drumming.tar.gz
https://s3.amazonaws.com/kinetics/600/val/alligator wrestling.tar.gz


In [12]:
import os
import urllib.request
import tarfile

# Create a directory to store the extracted videos
os.makedirs("kinetics_videos", exist_ok=True)

def download_and_extract_kinetics(file_path, num_files=5):
    """
    Reads URLs from the given file, downloads .tar.gz archives, and extracts videos.
    :param file_path: Path to the text file containing .tar.gz URLs.
    :param num_files: Number of .tar.gz files to process.
    """
    with open(file_path, "r") as f:
        urls = [line.strip().replace(" ", "%20") for line in f.readlines()[:num_files]]  # Fix spaces

    for url in urls:
        filename = os.path.basename(url).replace("%20", "_")  # Replace spaces with underscores
        save_path = os.path.join("kinetics_videos", filename)

        print(f"Downloading {filename}...")
        urllib.request.urlretrieve(url, save_path)

        print(f"Extracting {filename}...")
        with tarfile.open(save_path, "r:gz") as tar:
            tar.extractall("kinetics_videos")  # Extract into the same folder

        os.remove(save_path)  # Delete the .tar.gz file after extraction
        print(f"Done: {filename} ✅\n")

# Run the function to process 5 video categories
download_and_extract_kinetics("k600_val_paths.txt", num_files=5)

# List the extracted video files
print("\nExtracted files:")
!ls kinetics_videos


Extracting abseiling.tar.gz...
Done: abseiling.tar.gz ✅

Extracting acting_in_play.tar.gz...
Done: acting_in_play.tar.gz ✅

Extracting adjusting_glasses.tar.gz...
Done: adjusting_glasses.tar.gz ✅

Extracting air_drumming.tar.gz...
Done: air_drumming.tar.gz ✅

Extracting alligator_wrestling.tar.gz...
Done: alligator_wrestling.tar.gz ✅


Extracted files:
' '				 EDgbHrT9H0U_000065_000075.mp4	 PxddE1uuZzk_000113_000123.mp4
 08MqsEmN19s_000019_000029.mp4	 ee6nRkt3sF4_000133_000143.mp4	 qajBgS2lsDA_000003_000013.mp4
 0kz8c1GGOkI_000162_000172.mp4	 eHkBHH59-Js_000115_000125.mp4	 qFU1xp0aPlk_000002_000012.mp4
 0zxkyPzmVwY_000094_000104.mp4	 EJKStmtqqUk_000004_000014.mp4	 qqJisE6m6XI_000000_000010.mp4
 1gTjdDM-Q8g_000127_000137.mp4	 eZTQRUBQxqw_000031_000041.mp4	 QxZ5kYEJWxw_000004_000014.mp4
 1Nc-MMNvcro_000000_000010.mp4	 f2uK1SvWf5A_000005_000015.mp4	 qzF7Vpjmcas_000000_000010.mp4
 29j-viiLXSM_000011_000021.mp4	 f7Z6BmBkcq4_000006_000016.mp4	 r8rKbBOhP7Q_000029_000039.mp4
 2cPLjY5AWXU_00000

In [18]:
#Function to extract frames from video
import cv2
import torch
from torchvision import transforms
import os

# Define the frame extraction function
def extract_frames(video_path, num_frames=16):
    cap = cv2.VideoCapture(video_path)
    frames = []
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the frame interval to get `num_frames` equally spaced frames
    frame_interval = total_frames // num_frames

    count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_interval == 0:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB
            frame = transform(frame)  # Apply transformations
            frames.append(frame)
        count += 1
    cap.release()
    return torch.stack(frames)

# Preprocessing (resize, normalize, etc.)
transform = transforms.Compose([
    transforms.Resize((112, 112)),  # Resize the frames (for Kinetics models, typically 112x112)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [20]:
import cv2
import torch
from torchvision import transforms
from PIL import Image
import os

# Create the directory to save processed frames
output_folder = '/content/processed_frames'
os.makedirs(output_folder, exist_ok=True)

# Preprocessing transform (resize, normalize)
transform = transforms.Compose([
    transforms.Resize((112, 112)),  # Resize to 112x112 (common for Kinetics models)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def extract_frames(video_path, num_frames=16):
    cap = cv2.VideoCapture(video_path)
    frames = []
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the frame interval to get `num_frames` equally spaced frames
    frame_interval = total_frames // num_frames

    count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_interval == 0:
            # Convert the numpy array (OpenCV format) to a PIL image
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB
            frame = Image.fromarray(frame)  # Convert to PIL image
            frame = transform(frame)  # Apply transformations
            frames.append(frame)
        count += 1
    cap.release()
    return torch.stack(frames)

# Function to process all video files in the folder
def process_all_videos(video_folder, output_folder, num_frames=16):
    for video_file in os.listdir(video_folder):
        if video_file.endswith('.mp4'):  # Process only .mp4 files
            video_path = os.path.join(video_folder, video_file)
            frames = extract_frames(video_path, num_frames=num_frames)  # Extract frames from the video

            # Save frames as tensors (or as individual images)
            torch.save(frames, os.path.join(output_folder, f'{video_file}.pt'))  # Save frames as tensor
            print(f"Processed video: {video_file}")

# Process all videos in the `kinetics_videos` folder
process_all_videos('kinetics_videos', output_folder)

print("\nFrames extraction complete!")


Processed video: 9MTTl2lMkcY_000009_000019.mp4
Processed video: 8-zWZSctT7s_000241_000251.mp4
Processed video: eA6KqKuVj-I_000022_000032.mp4
Processed video: YqTT34PsD5c_000003_000013.mp4
Processed video: 7UOr6owN2i4_000176_000186.mp4
Processed video: iwm9xZpVjy8_000020_000030.mp4
Processed video: 2cPLjY5AWXU_000001_000011.mp4
Processed video: UFRaLM26S24_000016_000026.mp4
Processed video: DcwyDVypfYg_000000_000010.mp4
Processed video: DUN4TL-8D5s_000307_000317.mp4
Processed video: TOaz0kXfoO8_000013_000023.mp4
Processed video: cs4UM8n4-Jc_000311_000321.mp4
Processed video: ufmoEOVTggU_000024_000034.mp4
Processed video: czUp4fDypI8_000078_000088.mp4
Processed video: CSz0mcaxur4_000007_000017.mp4
Processed video: hasHFfny_X0_000008_000018.mp4
Processed video: rseBvhN6dio_000003_000013.mp4
Processed video: n4vBSHLnt6g_000050_000060.mp4
Processed video: xVam2K4WJ54_000007_000017.mp4
Processed video: GmL4wv70w7c_000165_000175.mp4
Processed video: zr1N60-zKh0_000193_000203.mp4
Processed vid

In [52]:
import os
import subprocess

# Initialize counters for extracted and skipped files
extracted_count = 0
skipped_count = 0

def extract_video_clip(video_file, start_time, end_time, output_folder="extracted_clips"):
    global extracted_count, skipped_count  # Access the counters

    # Check if the video file exists
    if not os.path.exists(video_file):
        print(f"Video file {video_file} does not exist. Skipping extraction.")
        skipped_count += 1  # Increment skipped count
        return  # Skip to the next one

    # Generate output path
    video_id = os.path.basename(video_file).split('.')[0]  # Extract video ID from file name
    output_file = os.path.join(output_folder, f"{video_id}_{start_time}_{end_time}.mp4")
    os.makedirs(output_folder, exist_ok=True)

    # FFmpeg command to extract the clip
    command = [
        "ffmpeg",
        "-i", video_file,
        "-ss", str(start_time),
        "-to", str(end_time),
        "-c", "copy",
        output_file
    ]

    try:
        result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"Extracted {output_file}")
        extracted_count += 1  # Increment extracted count
    except subprocess.CalledProcessError as e:
        print(f"Error occurred while extracting {video_file}. Details: {e.stderr.decode()}")
        print(f"Full command: {' '.join(command)}")
        print(f"stdout: {e.stdout.decode()}")
        print(f"stderr: {e.stderr.decode()}")

# Function to process all videos in the given folder
def process_all_videos(input_folder, start_time, end_time, output_folder="extracted_clips"):
    global extracted_count, skipped_count

    # Get a list of all .mp4 files in the input folder
    video_files = [f for f in os.listdir(input_folder) if f.endswith('.mp4')]

    if not video_files:
        print(f"No .mp4 files found in {input_folder}.")
        return

    # Process each video file
    for video_file in video_files:
        video_path = os.path.join(input_folder, video_file)
        extract_video_clip(video_path, start_time, end_time, output_folder)

    # After processing, print the results
    print(f"\nTotal videos extracted: {extracted_count}")
    print(f"Total videos skipped: {skipped_count}")


input_folder = "kinetics_videos"  # Folder where the video files are stored
start_time = 10  # Set start time (in seconds)
end_time = 20  # Set end time (in seconds)
process_all_videos(input_folder, start_time, end_time)


Extracted extracted_clips/9MTTl2lMkcY_000009_000019_10_20.mp4
Extracted extracted_clips/8-zWZSctT7s_000241_000251_10_20.mp4
Extracted extracted_clips/eA6KqKuVj-I_000022_000032_10_20.mp4
Extracted extracted_clips/YqTT34PsD5c_000003_000013_10_20.mp4
Extracted extracted_clips/7UOr6owN2i4_000176_000186_10_20.mp4
Extracted extracted_clips/iwm9xZpVjy8_000020_000030_10_20.mp4
Extracted extracted_clips/2cPLjY5AWXU_000001_000011_10_20.mp4
Extracted extracted_clips/UFRaLM26S24_000016_000026_10_20.mp4
Extracted extracted_clips/DcwyDVypfYg_000000_000010_10_20.mp4
Extracted extracted_clips/DUN4TL-8D5s_000307_000317_10_20.mp4
Extracted extracted_clips/TOaz0kXfoO8_000013_000023_10_20.mp4
Extracted extracted_clips/cs4UM8n4-Jc_000311_000321_10_20.mp4
Extracted extracted_clips/ufmoEOVTggU_000024_000034_10_20.mp4
Extracted extracted_clips/czUp4fDypI8_000078_000088_10_20.mp4
Extracted extracted_clips/CSz0mcaxur4_000007_000017_10_20.mp4
Extracted extracted_clips/hasHFfny_X0_000008_000018_10_20.mp4
Extracte

In [103]:
class_labels = pd.read_csv("/content/kinetics_600_labels.csv")

In [104]:
class_labels

,id,name
0,0,abseiling
1,1,acting in play
2,2,adjusting glasses
3,3,air drumming
4,4,alligator wrestling
...,...,...
595,595,writing
596,596,yarn spinning
597,597,yawning
598,598,yoga


#Using pre-trained model to predict

In [105]:
import torch
from torchvision import models, transforms
import os
import torch.nn.functional as F
import pandas as pd

# Load the ResNet3D model
model = models.video.r3d_18(pretrained=True)
model.eval()  # Switch to evaluation mode

# Load class labels
class_labels = pd.read_csv("/content/kinetics_600_labels.csv", index_col=0)  # Use 'id' as index

# Define transformation for video frames preprocessing
transform = transforms.Compose([
    transforms.Resize((112, 112)),  # Resize frames to the expected input size
    transforms.ToTensor(),          # Convert frames to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
])

# Load the frames tensor from .pt file
def load_frames(pt_file_path):
    return torch.load(pt_file_path)  # Load pre-saved tensor

# Prepare the video for model prediction
def prepare_video_for_model(frames_tensor):
    frames_tensor = frames_tensor.unsqueeze(0)  # Add batch dimension (batch_size, num_frames, channels, height, width)
    frames_tensor = frames_tensor.permute(0, 2, 1, 3, 4)  # [batch_size, channels, num_frames, height, width]
    return frames_tensor

# Predict the action for a given video tensor
def predict_action(frames_tensor):
    frames_tensor = prepare_video_for_model(frames_tensor)
    with torch.no_grad():
        outputs = model(frames_tensor)
        probabilities = F.softmax(outputs, dim=1)  # Apply softmax to get probabilities
        predicted_class = torch.argmax(probabilities, dim=1).item()  # Get class index
        predicted_label = class_labels.loc[predicted_class, 'name']  # Get label from CSV
        return predicted_label

# Loop through the .pt files and make predictions
folder_path = 'processed_frames'  # Your folder with .pt files
for pt_file in os.listdir(folder_path):
    if pt_file.endswith('.mp4.pt'):  # Check for your .pt video files
        pt_file_path = os.path.join(folder_path, pt_file)

        # Load frames and predict
        frames_tensor = load_frames(pt_file_path)
        predicted_label = predict_action(frames_tensor)
        print(f"Predicted action for {pt_file}: {predicted_label}")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
<ipython-input-105-7829f49c3fe0>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details

Predicted action for D4oh3U8EHMA_001286_001296.mp4.pt: bottling
Predicted action for EDgbHrT9H0U_000065_000075.mp4.pt: making sushi
Predicted action for _vzHWdv5F1U_000143_000153.mp4.pt: motorcycling
Predicted action for lxzkHipNiHM_000000_000010.mp4.pt: drumming fingers
Predicted action for f2uK1SvWf5A_000005_000015.mp4.pt: abseiling
Predicted action for 3vaoBMYK9no_000034_000044.mp4.pt: pumping gas
Predicted action for cs4UM8n4-Jc_000311_000321.mp4.pt: answering questions
Predicted action for YHCT_kezpfk_000063_000073.mp4.pt: bodysurfing
Predicted action for FJyH_uPLF7E_000018_000028.mp4.pt: battle rope training
Predicted action for CQzUU7-cVck_000006_000016.mp4.pt: card stacking
Predicted action for p1VKQa4N-hg_000000_000010.mp4.pt: abseiling
Predicted action for iwm9xZpVjy8_000020_000030.mp4.pt: playing ice hockey
Predicted action for yvhqfNATiAE_000057_000067.mp4.pt: bodysurfing
Predicted action for pj1OiSpSFI0_000022_000032.mp4.pt: acting in play
Predicted action for YwTc57Y3vj8_